# tf.estimator.Estimator, tf.estimator.RunConfig, tf.estimator.EstimatorSpec를 이용한 basic tutorial

In [534]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import layers
tf.__version__

'1.4.0'

<pre>tf.__version__  1.4.0부터 지원<pre>

In [535]:
dataX= [[1,2,3],[4,5,6],[7,8,9],[5,6,7]]
dataY = [[1],[2],[3],[4]]

In [554]:
# estimator의 model_fn에 들어갈 함수

#features: This is the first item returned from the input_fn passed to train, evaluate, and predict. This should be a single Tensor or dict of same.
#labels: This is the second item returned from the input_fn passed to train, evaluate, and predict. This should be a single Tensor or dict of same (for multi-head models). If mode is ModeKeys.PREDICT, labels=None will be passed. If the model_fn's signature does not accept mode, the model_fn must still be able to handle labels=None.
#mode: Optional. Specifies if this training, evaluation or prediction. See ModeKeys.
#params: Optional dict of hyperparameters. Will receive what is passed to Estimator in params parameter. This allows to configure Estimators from hyper parameter tuning.
#config: Optional configuration object. Will receive what is passed to Estimator in config parameter, or the default config. Allows updating things in your model_fn based on configuration such as num_ps_replicas, or model_dir.

#Returns: EstimatorSpec

#위와 같이 정의되어야 하고 return 은 EstimatorSpec을 return 해야 한다.

def model_fn(mode, features, labels,params):
    # features 은 나중에 estimator.train or estimator.predict or estimator.evaluate의 input_fn에 넣어준 구조체 이다.
    # estimator.train or estimator.predict or estimator.evaluate에서 넣어줄 input,label,param을 받아서
    # prediction, loss, train_op를 만든 후에 Estomator spec을 완성한후 return 해준다.
    
    #feature 처리
    if type(features) == dict:
        input_data = features["x"] # tensor object
    else:
        input_data = features
        
    print(input_data.shape)
    #graph
    output = tf.layers.dense(inputs=input_data,units=1,activation=None)
    print(output.shape)

    #mode처리
    #mode에 따라 다르게 estimatorSpec을 정의 할 수 있다.
    if mode == tf.estimator.ModeKeys.TRAIN:
        predictions = output
        loss = tf.reduce_mean(tf.pow(output-labels, 2))
        train_op = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)
        return tf.estimator.EstimatorSpec(
                  mode=mode,
                  loss=loss,
                  predictions=predictions,
                  train_op=train_op)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = output
        return tf.estimator.EstimatorSpec(
                mode=mode,
                predictions=predictions)
    

In [555]:
run_config = tf.contrib.learn.RunConfig(
        model_dir="./model_dir")

In [556]:
estimator = tf.estimator.Estimator(config=run_config,model_fn=model_fn,model_dir="./model_dir",params=None)

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1166166a0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './model_dir'}


INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1166166a0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './model_dir'}


In [557]:
estimator

In [558]:
my_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x":np.array(dataX,dtype=np.float32)},y=np.array(dataY,dtype=np.float32),shuffle=False,num_epochs=1000,batch_size=10)




In [559]:
estimator.train(input_fn = my_input_fn,hooks=None,steps=10,max_steps=None,saving_listeners=None)

(?, 3)
(?, 1)
INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Restoring parameters from ./model_dir/model.ckpt-0


INFO:tensorflow:Restoring parameters from ./model_dir/model.ckpt-0


INFO:tensorflow:Saving checkpoints for 0 into ./model_dir/model.ckpt.


...

INFO:tensorflow:loss = 0.420252, step = 0 (0.102 sec)


INFO:tensorflow:loss = 0.420362, step = 0 (0.105 sec)


INFO:tensorflow:loss = 0.420362, step = 0 (0.105 sec)


INFO:tensorflow:Loss for final step: 0.580446.


INFO:tensorflow:Loss for final step: 0.580446.


In [560]:
my_predict_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x":np.array([[1,2,3],[4,5,6]],dtype=np.float32)},shuffle=False,num_epochs=1)


result = estimator.predict(input_fn=my_predict_input_fn)

In [561]:
predictions = next(result)
predictions

(?, 3)
(?, 1)
INFO:tensorflow:Restoring parameters from ./model_dir/model.ckpt-0


INFO:tensorflow:Restoring parameters from ./model_dir/model.ckpt-0


array([ 0.96970105], dtype=float32)

In [562]:
predictions

array([ 0.96970105], dtype=float32)

In [563]:
p_list = list(predictions)

In [564]:
for p in p_list:
    print(p)

0.969701
